In [1]:
import sys
sys.path.append('../')
import os
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from instances.instance import *

problems are defined by: 
c, A, B (first stage jazz)

q_s, W_s, h_s, T_s, and clustering variables

In [2]:
gen = InvestmentGenerator('../stock_data.csv')
problem = gen.create_instance(40,400)

In [3]:
gen.means.mean()

0.06693744530339904

In [4]:
MP = gp.Model("MP")
MP.Params.outputFlag = 0
x = MP.addMVar((problem.s1_n_var,), name = "x")
eta = MP.addMVar((problem.k,), name = "eta", ub = problem.eta_bound, lb = -1*problem.eta_bound)

if problem.s1_direction == GRB.MAXIMIZE:
    MP.modelSense = GRB.MAXIMIZE
else:
    MP.modelSense = GRB.MINIMIZE
MP.setObjective(
    problem.c @ x + np.array([1/problem.k] * problem.k) @ eta 
)

c1 = MP.addConstr(
    problem.A @ x <= problem.b
)


cut_found = True
n_iters = 0
try: 
    while cut_found:
        cut_found = False
        q_vals = []
        if n_iters < 200:
            n_iters = n_iters + 1
        else:
            break
        MP.update()
        MP.optimize()
        LB = 0
        x_i = x.x
        eta_i = eta.x
        for s in range(problem.k):
            SP = gp.Model("SP")
            SP.Params.outputFlag = 0  # turn off output
            SP.Params.method = 1      # dual simplex
            y = SP.addMVar((problem.s2_n_var,), name = "y", obj = problem.q_list[s])
            if problem.s2_direction == GRB.MAXIMIZE:
                SP.modelSense = GRB.MAXIMIZE
            else:
                SP.modelSense = GRB.MINIMIZE
            res = SP.addConstr(
                problem.W_list[s] @ y <= problem.h_list[s] - (problem.T_list[s] @ x_i)
            )
            SP.optimize()
            Q = SP.ObjVal
            q_vals.append(Q)
            pi = res.Pi
            LB = LB + Q / problem.k
            if np.abs(Q - eta_i[s]) > 0.00001:
                if Q < eta_i[s]:
                    cut_found = True
                    p1 = pi @ problem.h_list[s]
                    p2 = pi @ problem.T_list[s]
                    if problem.s2_direction == GRB.MAXIMIZE:
                        MP.addConstr(
                            eta[s] <= p1 - gp.quicksum(p2[a] * x[a] for a in range(problem.s1_n_var))
                        )
                    else:
                        MP.addConstr(
                            eta[s] >= p1 - gp.quicksum(p2[a] * x[a] for a in range(problem.s1_n_var))
                        )        
        print(f"Iteration {n_iters}: LB = {LB}. UB = {MP.ObjVal}")
except:
    print(f"Errored out on iteration: {s}")



Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-07
Iteration 1: LB = 1467.5807075885389. UB = 560135.5071527364
Iteration 2: LB = 1467.5807075885389. UB = 1467.5807075885389


In [5]:
problem.h_list[s] - problem.T_list[s] @ x_i

array([1467.58070759,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ])

In [6]:
problem.other_info

{'initial_prices': array([140.34, 243.  , 141.83, 171.07, 237.12,  83.6 ,  56.83,  60.39,
        105.79,  28.56, 150.89,  36.22,  25.09,  97.48,  34.42, 104.06,
        106.91,  87.52,  62.18,  62.22,  77.88,  67.8 ,  65.38, 114.5 ,
         17.03,  43.7 ,  62.25,  39.77, 167.31, 196.34,  56.52,  75.5 ,
         54.63,  21.56, 179.01,  35.53,  70.62,  60.92, 133.09, 218.91]),
 'I_c': 0.6765888348963732,
 'I_u': 0.782931561215775,
 'R_l': 0.06693744530339904,
 'R_c': 0.052400916879864326,
 'capital': 28006.77535763658,
 'pct_changes': array([ 0.07925328,  0.11644458,  0.0343744 ,  0.04968094,  0.00857988,
        -0.03010197,  0.28704442, -0.2442796 ,  0.21433786, -0.40235831,
         0.00414199, -0.05442361, -0.14998152, -0.00644771,  0.36125914,
         0.75136913, -0.00224866,  0.31932536,  0.11531857,  0.31496828,
         0.26671705,  0.34442134,  0.17925758, -0.03644506, -0.1173439 ,
        -0.1775494 ,  0.17254713,  0.41205276,  0.19112815, -0.13648838,
         0.19200652, -

In [7]:
pd.DataFrame(problem.clust_vars).to_csv(
    'debug.csv'
)

array([[ 17.63,  50.38,  50.38, ...,  -1.35,  24.31,  51.37],
       [  6.39,  36.81,  11.01, ...,  11.96,  34.83,  12.79],
       [ 27.92,  60.5 , -43.39, ...,   7.41,  28.08, -30.62],
       ...,
       [ 14.42, -66.05,  -2.59, ...,   8.51,  23.56,  -0.52],
       [ 44.85,  55.05,  15.58, ...,  10.18, -38.78, -28.44],
       [ 11.12,  28.3 ,   4.88, ...,  18.06,  35.47,  15.02]])

In [8]:
q_vals

[1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075885468,
 1467.5807075

In [9]:
x.x

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])